In [3]:
import numpy as np

# 1. Download the Dataset and Understand the Format

400


# 2. Generate the Data Matrix and the Label vector

200


200


# 3. Split the Dataset into Training and Test sets

# 4. Classification using PCA

# 5. Classification Using LDA 

In [10]:
def center_function(d):
    m = np.mean(d, axis=0)
    return np.subtract(d, m)

In [12]:
def lda(data, l, dom, n_class, features):
  # Calculate the mean of feature for every class
  # Mean matrix is 10304x1 dim
  cdata = []
  for i in range(0, len(data), l):
    cdata.append(data[i:i+l])
  mean_c = []
  for i in range(n_class):
    mean_c.append(np.mean(cdata[i], axis=0))
  tmean = np.mean(data, axis=0) 

  # Calculate between-class scatter matrix
  # B matrix is 10304 x 10304
  bmat = np.zeros((features, features))
  tm = tmean[:, np.newaxis]
  for i in range(n_class):
    mc = mean_c[i][:, np.newaxis]
    sub = np.subtract(mc, tm)
    bmat += np.multiply(l, np.dot(sub, sub.T))

  # Center class matrix 
  # Zi matrix is 5 x 10304
  zmat = []
  for d in cdata:
    zmat.append(center_function(d))
  
  # Class scatter matrix 
  # S matrix is 10304 x 10304
  s_tot = np.zeros((features, features))
  for zi in zmat:
    s_tot += np.dot(zi.T, zi)
 
  # Compute dominant eigenvector
  eval, evec = np.linalg.eigh(np.dot(np.linalg.inv(s_tot), bmat))

  # Return the 39 dominant eigenvectors and the mean
  idx = eval.argsort()[::-1]
  eva = eval[idx]
  eve = evec[:,idx]
  dom_arr = eve[:,:dom]
  
  return dom_arr.T

In [13]:
pj_mat = lda(d1, 5, 39, 40, 10304)

In [14]:
pj_mat.shape

(39, 10304)

In [15]:
data1.shape

(200, 10304)

In [16]:
new_d1 = np.dot(data1, pj_mat.T)
new_d1.shape

(200, 39)

In [17]:
new_d2 = np.dot(data2, pj_mat.T)
new_d2.shape

(200, 39)

In [18]:
from sklearn.neighbors import KNeighborsClassifier

In [19]:
model_1_lda = KNeighborsClassifier(n_neighbors=1)
model_3_lda = KNeighborsClassifier(n_neighbors=3)
model_5_lda = KNeighborsClassifier(n_neighbors=5)
model_7_lda = KNeighborsClassifier(n_neighbors=7)

In [20]:
model_1_lda.fit(new_d1, label1)
model_3_lda.fit(new_d1, label1)
model_5_lda.fit(new_d1, label1)
model_7_lda.fit(new_d1, label1)

KNeighborsClassifier(n_neighbors=7)

In [25]:
pred_model_1_lda = model_1_lda.predict(new_d2)
pred_model_1_lda

array([ 1,  1,  1,  1, 32,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,
        7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10,  8, 11,
       11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14,
       14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17,
       18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21,
       21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24,
       24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28,
       28, 28, 37, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31,
       31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34,
        5, 15, 35, 25, 40, 36, 36, 36, 36, 17, 37, 37, 37, 37, 37, 38, 38,
       38, 23, 38, 39, 39, 39, 39, 39, 40,  5, 40, 40,  5])

In [26]:
label2

array([ 1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,
        4,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,  7,  7,  7,
        7,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10, 11,
       11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14,
       14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17,
       18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 21, 21,
       21, 21, 21, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 24,
       24, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 28,
       28, 28, 28, 28, 29, 29, 29, 29, 29, 30, 30, 30, 30, 30, 31, 31, 31,
       31, 31, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34,
       35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38, 38,
       38, 38, 38, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40])

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(label2, pred_model_1_lda)

0.945

# 6. Classifier Tuning 

# 7. Compare vs Non-Face Images

# 8. Bonus 